In [ ]:
#!pip install tweepy
#!pip install configparser
#!pip install wordcloud
#!pip install nltk
#!pip install textblob
#!pip install estnltk==1.7.1
#!pip install gensim 

In [9]:
import configparser
import tweepy
import pandas as pd
import re
import csv
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import time
import math
import datetime

## Authentication

In [ ]:
import configparser
config = configparser.RawConfigParser()
config.read('tweet.ini')
api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']
bearer_token =  config['twitter']['bearer_token']

## Tweepy setup

- API guide: https://dev.to/twitterdev/a-comprehensive-guide-for-using-the-twitter-api-v2-using-tweepy-in-python-15d9  
- Tweepy Documentation: https://docs.tweepy.org/en/stable/index.html   
- Twitter API v2 data dictionary: https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/tweet

In [11]:
import tweepy
client = tweepy.Client(bearer_token, wait_on_rate_limit = True)

auth = tweepy.OAuth2BearerHandler(bearer_token)
api = tweepy.API(auth, wait_on_rate_limit= True, retry_count=12, retry_delay=5, retry_errors=set([503, 500]))

In [12]:
valid_usernames = list(pd.read_csv('valid_twitter_usernames.csv')["0"])
len(valid_usernames)

232

In [13]:
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

# Leaders data

In [25]:
def queryFunction(usernames, start_interval, end_interval, csvWriter):
    idx = 0
    for username in usernames:
        if idx % 10 == 0:
            print("On user", str(idx) , "Current time is", str(datetime.datetime.now()))
            

        idx += 1
        
        query = 'from:' + username
        for response in tweepy.Paginator(client.search_all_tweets,
                                         query=query, 
                                         tweet_fields=['lang', 'public_metrics', 'created_at'],
                                         start_time=start_interval,
                                         end_time =end_interval,  
                                         max_results=500):
            time.sleep(1)
            if (response.data != None):
                for tweet in response.data:
                    tweetText = emoji_pattern.sub(r'', tweet.text)
                    retweet_count = 0
                    reply_count = 0
                    like_count = 0
                    quote_count = 0
                    impression_count = 0
                    if tweet.public_metrics:
                        retweet_count = tweet.public_metrics['retweet_count']
                        reply_count = tweet.public_metrics['reply_count']
                        like_count = tweet.public_metrics['like_count']
                        quote_count = tweet.public_metrics['quote_count']
                        impression_count = tweet.public_metrics['impression_count']

                    csvWriter.writerow([username, tweet.id, tweetText, tweet.lang, start_interval, end_interval, retweet_count, reply_count, like_count, quote_count, impression_count, tweet.created_at])


In [31]:
# set up csvWriter
header_row = ['user', 'tweet_id', 'tweet_text', 'tweet_lang', 'start', 'end', 'retweet_count', 'reply_count',
                        'like_count', 'quote_count', 'impression_count', 'created_at']

# define users and timeframes
usernames = valid_usernames

time_intervals = [
    #['2022-01-01T00:00:00.000Z','2022-01-31T23:59:00.000Z'], 
    #['2022-02-01T00:00:00.000Z','2022-02-28T23:59:00.000Z'], 
    #['2022-03-01T00:00:00.000Z','2022-03-31T23:59:00.000Z'], 
    #['2022-04-01T00:00:00.000Z','2022-04-30T23:59:00.000Z'],
    #['2022-05-01T00:00:00.000Z','2022-05-31T23:59:00.000Z'], 
    #['2022-06-01T00:00:00.000Z','2022-06-30T23:59:00.000Z'], 
    #['2022-07-01T00:00:00.000Z','2022-07-31T23:59:00.000Z'], 
    #['2022-08-01T00:00:00.000Z','2022-08-31T23:59:00.000Z'], 
    #['2022-09-01T00:00:00.000Z','2022-09-30T23:59:00.000Z'], 
    #['2022-10-01T00:00:00.000Z','2022-10-31T23:59:00.000Z'], 
    #['2022-11-01T00:00:00.000Z','2022-11-30T23:59:00.000Z'], 
    #['2022-12-01T00:00:00.000Z','2022-12-31T23:59:00.000Z'], 
    #['2023-01-01T00:00:00.000Z','2023-01-31T23:59:00.000Z'],
    ['2023-02-01T00:00:00.000Z','2023-02-28T23:59:00.000Z'],
]

#time_intervals = [['2022-02-21T23:59:00.000Z', '2022-03-31T23:59:00.000Z']] ## get missing data

# start counting time
start_time = time.time()

print('Total users', str(len(usernames)))
print('Total interval', str(len(time_intervals)))

# fetch the tweets
interval_counter = 0
for time_interval in time_intervals:
    print("----------------------------------------------------------")
    print("Starting with interval number " + str(interval_counter))
    print("Current time " + str(datetime.datetime.now()))
    print("----------------------------------------------------------")
    start_interval = time_interval[0]
    end_interval = time_interval[1]
    
    fileName = "new_data_24_02/interval_01_03_" + str(interval_counter) + "_from_" + str(start_interval).replace(":", "-") + "_until_" + str(end_interval).replace(":", "-") + ".csv"
    interval_counter += 1
    with open(fileName, 'a+', encoding="utf-8") as csvFile:
        csvWriter = csv.writer(csvFile)
        csvWriter.writerow(header_row)
        queryFunction(usernames, start_interval, end_interval, csvWriter)
    
    
    print("----------------------------------------------------------")
    print("Finsihed writing")
    print("Current time " + str(datetime.datetime.now()))
    
   
 # end counting time
end_time = time.time()                   
print("--- %s seconds ---" % (end_time - start_time))

Total users 11
Total interval 1
----------------------------------------------------------
Starting with interval number 0
Current time 2023-03-01 17:05:47.233417
----------------------------------------------------------
On user 0 Current time is 2023-03-01 17:05:47.237952
On user 10 Current time is 2023-03-01 17:06:01.706667
----------------------------------------------------------
Finsihed writing
Current time 2023-03-01 17:06:02.942298
--- 15.718170881271362 seconds ---


# Public data

In [ ]:
query = 'sõda' # war
for response in tweepy.Paginator(client.get_all_tweets_count, query=query, start_time='2022-02-23T16:12:22.000Z'):
    print(response)
    print(response.meta)

In [15]:
# test
query = 'ukraina põgenik'

tweets = client.search_all_tweets(query=query, 
                                  tweet_fields=['lang','author_id', 'created_at', 'public_metrics', 'geo', 'referenced_tweets' ], 
                                  expansions=["author_id"],
                                  user_fields=['username', 'location', 'id', 'protected', 'public_metrics', 'verified'],
                                  start_time='2022-02-23T16:12:22.000Z', 
                                  end_time = '2022-05-31T16:12:22.000Z',  
                                  max_results=10)

print(tweets)
print('---------')
print(tweets.data)
print('-------')
print(tweets.data[0].text)

#for tweet in tweets.data:
    #print(tweet.text)
    #if len(tweet.context_annotations) > 0:
    #    print(tweet.context_annotations)

Response(data=[<Tweet id=1513900133397975041 text='RT @viljararakas: Personali värbamise juures oli üks äärmiselt südamlik lugu. Päev tagasi jalutas hotelli peauksest sisse Ukraina põgenik,…'>, <Tweet id=1513899082431012867 text='RT @viljararakas: Personali värbamise juures oli üks äärmiselt südamlik lugu. Päev tagasi jalutas hotelli peauksest sisse Ukraina põgenik,…'>, <Tweet id=1513898200201142272 text='RT @viljararakas: Personali värbamise juures oli üks äärmiselt südamlik lugu. Päev tagasi jalutas hotelli peauksest sisse Ukraina põgenik,…'>, <Tweet id=1513897535592611843 text='Personali värbamise juures oli üks äärmiselt südamlik lugu. Päev tagasi jalutas hotelli peauksest sisse Ukraina põgenik, kes oli sõja alguses töötanud Kiievi tipphotellis, ning uuris tööle saamise võimalusi - täna liitus meiega. Julge pealehakkamine on enam kui pool võitu! 🇺🇦👏'>, <Tweet id=1506900160672059392 text='Venekeelset kooli eelistav Ukraina põgenik, kui kuuleb, et vene koolides võib ka putiniste olla

In [13]:
tweets.data[0].lang

'et'

In [71]:
tweets.data[0].referenced_tweets[0].type

'retweeted'

In [75]:
tweets.data[4].referenced_tweets == None

True

In [19]:
tweets.data[0].public_metrics

{'retweet_count': 3,
 'reply_count': 0,
 'like_count': 0,
 'quote_count': 0,
 'impression_count': 0}

In [69]:
tweets.includes['users'][0].public_metrics

14175289

In [57]:
len(tweets[1]['users'])

9

In [76]:
for t in tweets.data:
    print(t)
    
for u in tweets.includes['users']:
    print(u.verified)

RT @viljararakas: Personali värbamise juures oli üks äärmiselt südamlik lugu. Päev tagasi jalutas hotelli peauksest sisse Ukraina põgenik,…
RT @viljararakas: Personali värbamise juures oli üks äärmiselt südamlik lugu. Päev tagasi jalutas hotelli peauksest sisse Ukraina põgenik,…
RT @viljararakas: Personali värbamise juures oli üks äärmiselt südamlik lugu. Päev tagasi jalutas hotelli peauksest sisse Ukraina põgenik,…
Personali värbamise juures oli üks äärmiselt südamlik lugu. Päev tagasi jalutas hotelli peauksest sisse Ukraina põgenik, kes oli sõja alguses töötanud Kiievi tipphotellis, ning uuris tööle saamise võimalusi - täna liitus meiega. Julge pealehakkamine on enam kui pool võitu! 🇺🇦👏
Venekeelset kooli eelistav Ukraina põgenik, kui kuuleb, et vene koolides võib ka putiniste olla. Nendele inimestele tuleks kiiresti selgeks teha, mis vahe on eesti ja vene koolil, lisaks keelele. https://t.co/0E0iL1jNQ0
RT @jaakennuste: Võtsime täna tööle esimese Ukraina põgeniku. Mitte ainult seepära

In [5]:
def isRetweet(referenced_tweets):
    hasRetweet = False
    if referenced_tweets != None:
        for reference in referenced_tweets:
            if reference.type == 'retweeted':
                hasRetweet = True
                break
                
        return hasRetweet
    else:
        return False
        

In [6]:
var_tweet_fields = ['lang','author_id', 'created_at', 'public_metrics', 'geo', 'referenced_tweets' ]
var_user_fields = ['username', 'id', 'public_metrics']

header_row_tweet = ['tweet_id', 'author_id', 'tweet_text', 'tweet_lang','created_at', 'geo', 'isRetweet', 'retweet_count', 'reply_count',
                        'like_count', 'quote_count', 'impression_count']

header_row_user = ['id', 'username', 'followers_count', 'following_count',
                        'tweet_count', 'listed_count']

def queryTweets(query,csvTweetWriter, csvUserWriter):
    for response in tweepy.Paginator(client.search_all_tweets,
                                      query=query, 
                                      tweet_fields=var_tweet_fields, 
                                      expansions=["author_id"],
                                      user_fields=var_user_fields,
                                      start_time='2022-01-01T00:00:00.000Z', 
                                      end_time = '2023-01-31T23:59:00.000Z',  
                                      max_results=500):
        time.sleep(1)
        if (response.data != None):
            for tweet in response.data:
                tweetText = emoji_pattern.sub(r'', tweet.text)
                retweet_count = 0
                reply_count = 0
                like_count = 0
                quote_count = 0
                impression_count = 0
                retweet = isRetweet(tweet.referenced_tweets)
                
                if tweet.public_metrics:
                    retweet_count = tweet.public_metrics['retweet_count']
                    reply_count = tweet.public_metrics['reply_count']
                    like_count = tweet.public_metrics['like_count']
                    quote_count = tweet.public_metrics['quote_count']
                    impression_count = tweet.public_metrics['impression_count']
                csvTweetWriter.writerow([tweet.id, tweet.author_id, tweetText, tweet.lang, tweet.created_at, tweet.geo, retweet, retweet_count, reply_count, like_count, quote_count, impression_count])
        
        if ('users' in response[1]):
            for user in response.includes['users']:
                followers_count = 0
                following_count = 0
                tweet_count = 0
                listed_count = 0
                if user.public_metrics:
                    if followers_count in user.public_metrics:
                        followers_count = user.public_metrics['followers_count']
                    if following_count in user.public_metrics:
                        following_count = user.public_metrics['following_count']
                    if tweet_count in user.public_metrics:
                        tweet_count = user.public_metrics['tweet_count']
                    if listed_count in user.public_metrics:
                        listed_count = user.public_metrics['listed_count']
                    
                    
                    
                
                csvUserWriter.writerow([user.id, user.username,followers_count, following_count,
                        tweet_count, listed_count])


In [29]:
##keywords_est = ['ukraina', 'ua', 'ukrainlane', 'põgenik', 'venemaa', 'sõda', 'konflikt', 'lääs', 'lääneriigid', 'agressioon', 
##                'agressor','nato', 'sõjapõgenik', 'pagulane', 'migrant']

#uasoda = ['ukraina sõda', 'ukraina sõja', 'ukraina sõjas', 'ukraina sõjast', 'ukraina sõjaks', 'ukraiana sõjalt', 'ukraina sõjal', 'ukraina sõjale','ukraina sõjani', 'ukraina sõjaga']
#uapogenik = ['ukraina põgenik', 'ukraina põgeniku', 'ukraina põgenikku', 'ukraina põgenikule', 'ukraina põgenikust', 'ukraina põgenikuks', 'ukraina põgenikuna']
#uaother = ['ukraina venemaa','ukraina konflikt', 'ukraina aggressioon', 'venemaa agressioon']
#uaotherpogenik = ['ukraina pagulane', 'ukraina migrant', 'ukraina sõjapõgenik', 'ukraina sõjapõgeniku']

#venemaa = ['venemaa agressioon', 'venemaa agressor', 'venemaa agressiooni', 'venemaa putin', 'venemaa sõda', 'venemaa erioperatsioon', 'venemaa putini', 'venemaa putinil']

#ua_positive = [ 'ukraina abi', 'ukraina toetus', 'ukraina koostöö', 'ukraina toetamine']

giant = ['ukraina', 'ua', 'ukrainlane', 'põgenik', 'venemaa', 'sõda', 'konflikt', 'lääs', 'lääneriigid', 'agressioon', 
                'agressor','nato', 'sõjapõgenik', 'pagulane', 'migrant']

In [ ]:
words = []
#words.concat(uasoda).concat(uapogenik).concat(uaother).concat(uaotherpogenik).concat(venemaa).concat(ua_positive)
words = [] + uaotherpogenik + venemaa + ua_positive + giant

In [7]:
giant2 = ['ukrainlane', 'põgenik', 'sõjapõgenik', 'sõda', 'lääs', 'lääneriigid','pagulane ukraina',
          'venemaa','agressioon']

In [8]:
venemaa2 = ['venemaa sanktsioon', 'venemaa ründama', 'venemaa ründamine', 'venemaa agressor','venemaa ründas', 'venemaa rünnak',
           'venemaa konflikt', 'putini sõda']

ukraina = ['ukraina liitlane', 'ukraina sanktsioon', 'ukraina ründama', 'ukraina ründas', 'ukraina rünnak', 'ukraina julgeolek',
          'lääs ukraina', 'immigratsioon ukraina', 'ukraina agressor', 'ukraina elu eesti', 'varjupaiga ukraina', 
             'varjupaik ukraina']

uainimene = ['ukraina inimene', 'ukraina inimese', 'ukraina inimeste', 'ukraina inimestel', 'ukraina inimestega', 
             'ukraina inimestest', 'ukraina inimest', 'ukraina inimesele', 'ukraina inimesena', 'ukraina inimesega']



   

In [9]:
words2 = []
#words.concat(uasoda).concat(uapogenik).concat(uaother).concat(uaotherpogenik).concat(venemaa).concat(ua_positive)
words2 = [] + venemaa2 + ukraina + uainimene + giant2

In [12]:
# ukraina and sõda keywords are half done - crashed for some reasons
words3 = ['venemaa putin', 'ukraina putin', 'ukraina putini', 'sõda putin', 'sõjakuritegu', 'sõjakuriteod', 'ukraina erioperatsioon',
          'ukraina vabastamine', 'ukraina vaba',
          'lääneriigid','pagulane ukraina','lääs', 'venemaa','agressioon']

In [26]:
words4 = ["eesti ukraina", "eesti venemaa", "eesti putin", "\"sõda\" lang:et", "ukraina lang:et"]

In [28]:
words5 = ["\"sõda\" lang:et", "ukraina lang:et"]

In [29]:
start_time = time.time()

print('Total keywords', str(len(words5)))

# fetch the tweets
keyword_counter = 1
for keyword in words5: # word lists here
    print("----------------------------------------------------------")
    print("Starting with keyword number " + str(keyword_counter) +": " + str(keyword))
    print("Current time " + str(datetime.datetime.now()))
    print("----------------------------------------------------------")
    
    tweetFileName = "keywords_data/tweet_keyword_" + keyword.replace('"', '') + ".csv"
    userFileName = "users_data/user_keyword_" + keyword.replace('"', '') + ".csv"
    keyword_counter += 1
    
    tweet_file = open(tweetFileName, 'a+', encoding="utf-8")
    user_file = open(userFileName, 'a+', encoding="utf-8")
    
    tweetWriter = csv.writer(tweet_file)
    userWriter = csv.writer(user_file)
    
    queryTweets(keyword,tweetWriter, userWriter)

    tweet_file.close()
    user_file.close()
    
    print("----------------------------------------------------------")
    print("Finsihed writing")
    print("Current time " + str(datetime.datetime.now()))
    
   
 # end counting time
end_time = time.time()                   
print("--- %s seconds ---" % (end_time - start_time))

Total keywords 2
----------------------------------------------------------
Starting with keyword number 1: "sõda" lang:et
Current time 2023-02-05 22:54:00.654513
----------------------------------------------------------
----------------------------------------------------------
Finsihed writing
Current time 2023-02-05 22:56:10.576553
----------------------------------------------------------
Starting with keyword number 2: ukraina lang:et
Current time 2023-02-05 22:56:10.576553
----------------------------------------------------------
----------------------------------------------------------
Finsihed writing
Current time 2023-02-05 22:59:37.082870
--- 336.42835688591003 seconds ---


# Municipality leaders

...because they were left out in the first run

In [26]:
users = pd.read_excel('./leaders_df/KOVid_15092022.xlsx', 'Sheet2')

In [23]:
twitter_names = []
follower_count = []
tweet_count = []
last_tweet_times = []

# start counting time
start_time = time.time()
print("Total rows", len(users))
for i,row in users.iterrows():
    current_name = ''
    current_follower_count = None
    current_tweet_count = None
    current_last_tweet_time = None
    
    if (i % 20 == 0):
        print("Currently done rows count", str(i))

    # This .items(1) takes the first match
    for cursor_user in tweepy.Cursor(api.search_users, q=row['nimi']).items(1):
        current_name = cursor_user.screen_name
        current_follower_count = cursor_user.followers_count
        current_tweet_count = cursor_user.statuses_count
        if hasattr(cursor_user, 'status'):
            current_last_tweet_time = cursor_user.status.created_at
    
    twitter_names.append(current_name)
    follower_count.append(current_follower_count)
    tweet_count.append(current_tweet_count)
    last_tweet_times.append(current_last_tweet_time)

# end counting time
end_time = time.time()                   
print("--- %s seconds ---" % (end_time - start_time))

Total rows 156
Currently done rows count 0
Currently done rows count 20
Currently done rows count 40
Currently done rows count 60
Currently done rows count 80
Currently done rows count 100
Currently done rows count 120
Currently done rows count 140
--- 61.87808704376221 seconds ---


In [29]:
df_users = users
df_users['twitter_usernames'] = twitter_names
df_users['twitter_followers'] = follower_count
df_users['twitter_tweet_count'] = tweet_count
df_users['last_tweet_time'] = last_tweet_times

In [34]:
years = []

for i, row in df_users.iterrows():
    if pd.isna(row['last_tweet_time']):
        years.append(row['last_tweet_time'])
        continue
    element = datetime.datetime.strptime(str(row['last_tweet_time']),"%Y-%m-%d %H:%M:%S%z")
    years.append(element.year)

In [ ]:
df_users['last_tweet_year'] = years

In [36]:
valid_usernames = []

for idx, user in df_users.iterrows():
    username = user['twitter_usernames']
    tweet_count = user['twitter_tweet_count']
    last_tweet_year = user['last_tweet_year']


    user_has_no_username = username == '' or pd.isna(username) # check if twitter username exists for a certain person
    user_has_no_tweets = tweet_count == 0 # check if user has tweets
    user_has_not_tweeted_lately = pd.isna(last_tweet_year) or int(last_tweet_year) < 2022 # check if last tweet made in 2022
    if user_has_no_username or user_has_no_tweets or user_has_not_tweeted_lately:
        continue
        
    valid_usernames.append(username)

In [37]:
df_users.to_csv('kov_useres_twitter_df.csv', encoding="utf-8")

In [40]:
pd.DataFrame(valid_usernames).to_csv('kov_cleaned_twitter_usernames.csv', encoding="utf-8")